In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ && cd /root/NVIDIA_CUDA-11.2_Samples/

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-q1pepg7l
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-q1pepg7l
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=e6fa99e1a9998cb3754a4688e462becbe9cb8e8d1b8a241bf7be97e98513da5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-2o0tboqa/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin
cre

In [ ]:
%%cuda --name my_curand.cu

#include <cstdlib>
#include <curand.h>
#include <cublas_v2.h>
#include <iostream>
#include <ctime>

using namespace std;

void GPU_fill_rand(float *A, int nr_rows_A, int nr_cols_A) {
  curandGenerator_t prng;
  curandCreateGenerator(&prng, CURAND_RNG_PSEUDO_DEFAULT);
  curandSetPseudoRandomGeneratorSeed(prng, (unsigned long long) clock());
  curandGenerateUniform(prng, A, nr_rows_A * nr_cols_A);
}

// C(m,n) = A(m,k) * B(k,n)
void gpu_blas_mmul(const float *A, const float *B, float *C, const int m, const int k, const int n) {
  int lda=m,ldb=k,ldc=m;
  const float alf = 1;
  const float bet = 0;
  const float *alpha = &alf;
  const float *beta = &bet;
  cublasHandle_t handle;
  cublasCreate(&handle);
  cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);
  cublasDestroy(handle);
}

void print_matrix(float* matrix, int rows, int cols) {
  for(int i = 0; i < rows; ++i){
    for(int j = 0; j < cols; ++j){
      printf("%f ", matrix[i + j * rows]);
    }
    printf("\n");
  }
  printf("\n");
}

void fill_matrix(float* matrix, int rows, int cols) {
  for(int i = 0; i < rows; ++i){
    for(int j = 0; j < cols; ++j) {
      matrix[i + j * rows] = (rand() + 0.0) / RAND_MAX;
    }
  }
}

int main() {
  int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B, nr_rows_C, nr_cols_C;
  nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = nr_rows_C = nr_cols_C = 6000;
  float *h_A = (float *)malloc(nr_rows_A * nr_cols_A * sizeof(float));
  float *h_B = (float *)malloc(nr_rows_B * nr_cols_B * sizeof(float));
  float *h_C = (float *)malloc(nr_rows_C * nr_cols_C * sizeof(float));
  float *h_C_ = (float *)malloc(nr_rows_C * nr_cols_C * sizeof(float));

  float *d_A, *d_B, *d_C;
  cudaMalloc(&d_A,nr_rows_A * nr_cols_A * sizeof(float));
  cudaMalloc(&d_B,nr_rows_B * nr_cols_B * sizeof(float));
  cudaMalloc(&d_C,nr_rows_C * nr_cols_C * sizeof(float));

  GPU_fill_rand(d_A, nr_rows_A, nr_cols_A);
  GPU_fill_rand(d_B, nr_rows_B, nr_cols_B);

  //fill_matrix(h_A, nr_rows_A, nr_cols_A);
  //fill_matrix(h_B, nr_rows_B, nr_cols_B);

  double start = clock();
  cudaMemcpy(d_A, h_A, nr_rows_A * nr_cols_A * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nr_rows_B * nr_cols_B * sizeof(float), cudaMemcpyHostToDevice);
  double end = clock();

  cudaMemcpy(h_A, d_A, nr_rows_A * nr_cols_A * sizeof(float),cudaMemcpyDeviceToHost);
  cudaMemcpy(h_B, d_B, nr_rows_B * nr_cols_B * sizeof(float),cudaMemcpyDeviceToHost);

  printf("A = \n");
  print_matrix(h_A, 5, 5);

  printf("B = \n");
  print_matrix(h_B, 5, 5);

  double start_ = clock();
  gpu_blas_mmul(d_A, d_B, d_C, nr_rows_A, nr_cols_A, nr_cols_B);
  double end_ = clock();

  cudaMemcpy(h_C,d_C,nr_rows_C * nr_cols_C * sizeof(float),cudaMemcpyDeviceToHost);
  printf("C = \n");
  print_matrix(h_C, 5, 5);

  cout << "\nTime = " << (end - start) / CLOCKS_PER_SEC << endl;

  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  free(h_A);
  free(h_B);
  free(h_C);
  return 0;
}


'File written in /content/src/my_curand.cu'

In [ ]:
%%cuda --name my_curand.cu

#include <iostream>
#define Q 12
#define NMAX 8200000

int main(int argc, char* argv[]) {
    double a[NMAX], sum[NMAX], b[NMAX];
    int n;
    n = NMAX;

    // инициализация массивов
    for (int j = 0; j < n; j++) {
        a[j] = 1;
        b[j] = 1;
    }

    //суммирование векторов
    double st_time, end_time = 0;
    //последовательная программа
    for (int timeCount = 0; timeCount < 12; timeCount++) {
        st_time = clock();
        for (int k = 0; k < n; k++) {
            for (int j = 0; j < Q; j++) {
                sum[k] = a[k] + b[k];
            }
        }
        end_time += clock() - st_time;
    }

    printf("\nTime: %f ", end_time / 12);
    printf("\nsum[0]=%f, a[0]=%f, b[0]=%f", sum[0], a[0], b[0]);
    printf("\nsum[1]=%f, a[1]=%f, b[1]=%f", sum[1], a[1], b[1]);
    printf("\nsum[1]=%f, a[1]=%f, b[1]=%f", sum[1], a[1], b[1]);


}


'File written in /content/src/my_curand.cu'

In [ ]:
!nvcc -o /content/src/my_curand /content/src/my_curand.cu -lcurand -lcublas

In [ ]:
!/content/src/my_curand